<a href="https://colab.research.google.com/github/aish793/Sandbox/blob/master/Deep%20Learning/Language%20modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IDL Assignment 5 - Language Modeling & Recurrent Neural Networks**

### Team Members


1.   Amar Shivaram - 226015
2.   Manish Bhandari - 226011
3.   Aishwarya Jauhari - 226084






## **Assigning Tensorflow version and importing the libraries required for the tasks**


In [ ]:
%tensorflow_version 2.x

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os

tf.__version__

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


'2.2.0'

In [ ]:
os.getcwd()
os.chdir("/content/drive/My Drive/Colab Notebooks/IDL /IDL Assignments/Assignment helper files") 


## Preprocess the text data

In [ ]:
!python prepare_data.py shakespeare_input.txt skp

**total serialized seq is 22981**

**Loading the data from skp.tfrecords and skp_vocab**

The files mentioned are the output obtained after running the program *prepare_data.py* for the Shakespeare data. These files are loaded as data using tf.data and create a vocabulary dictionary 

**Note:** The vocab contains elements as dict with (key,val) as (character, index). Reverse mapping is done and stored as ind_to_ch which has (key,val) as (index,character)

In [ ]:
from prepare_data import parse_seq
import pickle

# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("skp.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
data = data.map(lambda x: parse_seq(x, 200))

# a map from characters to indices
vocab = pickle.load(open("skp_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(vocab)
print(vocab_size)

print("Indices to char")
print(ind_to_ch)

{'h': 1, 'L': 2, 'b': 3, 's': 4, 'n': 5, 'q': 6, 'Q': 7, '3': 8, 'H': 9, 'Y': 10, 'w': 11, 'P': 12, '-': 13, 'g': 14, "'": 15, 'N': 16, 'f': 17, ';': 18, 'j': 19, 'v': 20, 'E': 21, 'x': 22, 'J': 23, 'S': 24, 'O': 25, 'K': 26, 'G': 27, 'a': 28, 'k': 29, 'o': 30, 'R': 31, 'i': 32, ':': 33, 'u': 34, 'M': 35, 'U': 36, ' ': 37, 'B': 38, 'C': 39, ',': 40, 't': 41, 'X': 42, 'r': 43, '$': 44, 'm': 45, 'z': 46, 'y': 47, '.': 48, 'l': 49, '?': 50, 'D': 51, ']': 52, 'c': 53, '!': 54, 'p': 55, '&': 56, 'Z': 57, 'I': 58, '[': 59, 'd': 60, 'W': 61, 'F': 62, '\n': 63, 'e': 64, 'T': 65, 'A': 66, 'V': 67, '<S>': 0}
68
Indices to char
{1: 'h', 2: 'L', 3: 'b', 4: 's', 5: 'n', 6: 'q', 7: 'Q', 8: '3', 9: 'H', 10: 'Y', 11: 'w', 12: 'P', 13: '-', 14: 'g', 15: "'", 16: 'N', 17: 'f', 18: ';', 19: 'j', 20: 'v', 21: 'E', 22: 'x', 23: 'J', 24: 'S', 25: 'O', 26: 'K', 27: 'G', 28: 'a', 29: 'k', 30: 'o', 31: 'R', 32: 'i', 33: ':', 34: 'u', 35: 'M', 36: 'U', 37: ' ', 38: 'B', 39: 'C', 40: ',', 41: 't', 42: 'X', 43: '

In [ ]:
data

<MapDataset shapes: (200,), types: tf.int32>

In [ ]:
## Declare the sizes of batch, shuffle and repeat

SHUFFLE_SIZE = 1000
BATCH_SIZE = 128
REPEAT_TIMES = 20

In [ ]:
def batch_shuffle_repeat(data):


    data = data.shuffle(SHUFFLE_SIZE)
    data = data.padded_batch(BATCH_SIZE, padded_shapes=None,drop_remainder=False)   
    data = data.repeat(REPEAT_TIMES)


    return data

In [ ]:
dataset = batch_shuffle_repeat(data)


In [ ]:
for x in dataset.take(1):
  print(tf.shape(x))
  print(type(x))
  print(repr(tf.shape(x)))
  print(x)

tf.Tensor([128 200], shape=(2,), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
<tf.Tensor: shape=(2,), dtype=int32, numpy=array([128, 200], dtype=int32)>
tf.Tensor(
[[ 0 64 37 ... 29 64  4]
 [ 0  5 37 ... 30 34 43]
 [ 0 64 43 ... 37 55 64]
 ...
 [ 0 47 40 ... 64 20 30]
 [ 0 37 32 ...  1 47 40]
 [ 0 34 53 ... 11  1 32]], shape=(128, 200), dtype=int32)


In [ ]:
n_h = 512

## w_xh is input to hidden weight --> known as U from the literature
## w_hh is hidden to hidden weights --> known as W from the literature
## w_ho is hidden to output weights --> known as V from the literature
## b_h and b_o are the biases at the hidden layer and output layer


w_xh = tf.Variable(tf.initializers.glorot_uniform()([vocab_size,n_h]))

w_hh = tf.Variable(tf.initializers.glorot_uniform()([n_h,n_h]))
b_h = tf.Variable(tf.zeros([n_h]))

w_ho = tf.Variable(tf.initializers.glorot_uniform()([n_h,vocab_size]))
b_o = tf.Variable(tf.zeros([vocab_size]))

variables = [w_xh,w_hh,b_h,w_ho,b_o]

In [ ]:
opt = tf.optimizers.Adam()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


@tf.function
def rnn_sequence(batch_data):
    with tf.GradientTape() as tape:
        h_t = tf.zeros([tf.shape(batch_data)[0],n_h])
        loss = tf.TensorArray(tf.float32,size=tf.shape(batch_data)[1]-1)

        for timestep in tf.range(tf.shape(batch_data)[1]-1):
            x_t = tf.one_hot(batch_data[:,timestep],vocab_size)
            h_t = tf.nn.tanh(tf.matmul(x_t,w_xh) + tf.matmul(h_t,w_hh) + b_h)
            logits = tf.matmul(h_t,w_ho) + b_o

            local_loss = loss_fn(batch_data[:,timestep+1],logits)

            loss = loss.write(timestep, local_loss)
        loss = loss.stack()

        batch_loss = tf.reduce_mean(loss)
        
    
    grads = tape.gradient(batch_loss, variables)
    opt.apply_gradients(zip(grads, variables))

    return batch_loss,h_t


In [ ]:
##### DONT RUN THIS BLOCK .. THIS IS JUST FOR TRIAL PURPOSES


###############################################################################################################################################
opt = tf.optimizers.Adam()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

"""
        tf.print(tf.shape(batch_data))
        tf.print(type(batch_data))
        tf.print(repr(tf.shape(batch_data)))
        tf.print(batch_data)

        for i in tf.range(tf.shape(batch_data)[1]-1):
          tf.print(i)
"""



@tf.function
def rnn_try(batch_data):
    with tf.GradientTape() as tape:
        h_t = tf.zeros([tf.shape(batch_data)[0],n_h])
        loss = tf.TensorArray(tf.float32,size=tf.shape(batch_data)[1]-1)
        

        for timestep in tf.range(tf.shape(batch_data)[1]-1):
            tf.print("="*100)
            tf.print(timestep)

            x_t = tf.one_hot(batch_data[:,timestep],vocab_size)
            tf.print(tf.shape(x_t))
            tf.print(type(x_t))
            tf.print(repr(tf.shape(x_t)))



            h_t = tf.nn.tanh(tf.matmul(x_t,w_xh) + tf.matmul(h_t,w_hh) + b_h)
            tf.print(tf.shape(h_t))
            tf.print(type(h_t))
            tf.print(repr(tf.shape(h_t)))


            logits = tf.matmul(h_t,w_ho) + b_o
            tf.print(tf.shape(logits))
            tf.print(type(logits))
            tf.print(repr(tf.shape(logits)))


            local_loss = loss_fn(batch_data[:,timestep+1],logits)
            tf.print(tf.shape(local_loss))
            tf.print(type(local_loss))
            tf.print(repr(tf.shape(local_loss)))


            loss = loss.write(timestep, local_loss)

        loss = loss.stack()
        tf.print(tf.shape(loss))
        tf.print(type(loss))
        tf.print(repr(tf.shape(loss)))





In [ ]:
import time

start = time.time()


batch_nr = 0
for batch_data in dataset:
      batch_start = time.time()
      batch_nr = batch_nr+1
      batch_loss,h_t = rnn_sequence(batch_data)
      batch_stop = time.time()
#      rnn_try(batch_data)
      print("Batch Number: {} Loss: {} Time taken: {}".format(batch_nr,batch_loss,batch_stop-batch_start))
#      if not steps % 100:
#          train_acc_metric(lbl_batch, logits)
#          acc = train_acc_metric.result()
#          print("Loss: {} Accuracy: {}".format(loss, acc))
#          train_acc_metric.reset_states()

stop = time.time()
print("took {} seconds\n".format(stop-start))

Batch Number: 1 Loss: 4.221291542053223 Time taken: 3.126525640487671
Batch Number: 2 Loss: 3.833584785461426 Time taken: 0.38123464584350586
Batch Number: 3 Loss: 3.5180790424346924 Time taken: 0.3758275508880615
Batch Number: 4 Loss: 3.583409309387207 Time taken: 0.3216116428375244
Batch Number: 5 Loss: 3.39400315284729 Time taken: 0.2955586910247803
Batch Number: 6 Loss: 3.4939541816711426 Time taken: 0.2938830852508545
Batch Number: 7 Loss: 3.3784704208374023 Time taken: 0.3131701946258545
Batch Number: 8 Loss: 3.363752603530884 Time taken: 0.2981090545654297
Batch Number: 9 Loss: 3.385077953338623 Time taken: 0.3008840084075928
Batch Number: 10 Loss: 3.3499796390533447 Time taken: 0.2968623638153076
Batch Number: 11 Loss: 3.3489768505096436 Time taken: 0.2958827018737793
Batch Number: 12 Loss: 3.3338100910186768 Time taken: 0.3241133689880371
Batch Number: 13 Loss: 3.311328887939453 Time taken: 0.3471994400024414
Batch Number: 14 Loss: 3.3387887477874756 Time taken: 0.348881244659

In [ ]:
characters = [0,5,6]
print(characters)

print(characters[-1:])

[0, 5, 6]
[6]


#### **Using np.random.choice**

In [ ]:
num_generate = 2500


text_generated = []
#temp = 1.5
#h_t = tf.zeros([1,n_h])
start_char = '<S>'
choice = []
val = vocab[start_char]
choice.append(val)

for i in range(num_generate):
  x_t = tf.one_hot(choice[-1:],depth = vocab_size)
  a = (tf.matmul(x_t,w_xh))+ (tf.matmul(h_t, w_hh)) + b_h
  #print(type(a))
  h_t = tf.nn.tanh(a)
  logits = (tf.matmul(h_t, w_ho)) + b_o
  #print(logits)
  predictions = tf.nn.softmax(logits)
  #predictions = predictions[0,:]
  # predictions = tf.squeeze(predictions, 0)
  #print("----------------------"*3)
  #print(predictions.shape)
  #print(predictions)
  #print(predictions.numpy().shape)
  #predictions = predictions / temp

  ## The predictions is a tensor 
  predictions = predictions.numpy()[0]
  #predictions = np.array(predictions)
  #print(predictions.shape)
  #print(predictions)
  char_choice = np.random.choice(vocab_size, p = predictions)
  #print(char_choice)
  choice.append(char_choice)
  #print(choice)
  #predictions = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
  #print(predictions)
  start_char = ind_to_ch[char_choice]
  #print(start_char)
  #print(input_eval)
  text_generated.append(start_char)
  
print("".join(text_generated))
print("="*90)


en. Who, ave adow a whole, the weam when yod a gear'l stand it sung, if it will so will dread desires, of is death
Is thou fell wear father,
But the grave inquest-to it again the baich's surper unto this honouraigs so swon
Next be the law and solered do is ne'er rearons and sworn, acch medie: I hear him: loviss wse decelseng my fous for unjmyed, they and oderry aftat of his no magoor tequit fount, thou now defey in his was one mather time should you known, would to pear thy grandat this gown, mansten Duke of seautance as thou art lie to:
I'll fable true to heart non misere gave of right
May your rejest and of it is He Isterm'd in sead my Pains?
Whise is thou not the devil truth. 'sid it adomen's own, hos thr foreless dead as how deverse make ary
overades of man, and you as
that thus ent his toss in grands 'tis im'ting and constate
Iven's as for mine in my
Francers.

KING JOAN:
If you? will be a sufford passion.
O hard for a seemirits, by this miscreed.

CARDINAL:
I am nature and sis br

#### **Using np.argmax**



In [ ]:
num_generate = 2500


text_generated = []
#temp = 1.5
h_t = tf.zeros([1,n_h])
start_char = '<S>'
choice = []
val = vocab[start_char]
choice.append(val)

for i in range(num_generate):
  x_t = tf.one_hot(choice[-1:],depth = vocab_size)
  a = (tf.matmul(x_t,w_xh))+ (tf.matmul(h_t, w_hh)) + b_h
  #print(type(a))
  h_t = tf.nn.tanh(a)
  logits = (tf.matmul(h_t, w_ho)) + b_o
  #print(logits)
  predictions = tf.nn.softmax(logits)
  #predictions = predictions[0,:]
  # predictions = tf.squeeze(predictions, 0)
  #print("----------------------"*3)
  #print(predictions.shape)
  #print(predictions.numpy().shape)
  #predictions = predictions / temp
  predictions = predictions.numpy()[0]
  #print(predictions.shape)
  #print(predictions)
  char_choice = np.argmax(predictions)
  #print(char_choice)
  #char_choice = np.random.choice(vocab_size, p = predictions)
  #print(char_choice)
  choice.append(char_choice)
  #predictions = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
  #print(predictions)
  start_char = ind_to_ch[char_choice]
  #print(start_char)
  #print(input_eval)
  text_generated.append(start_char)
  
print("".join(text_generated))

 the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange to see the see in and the strange t

#### **Try outs**

In [ ]:
dummy_char = '<S>'
ch = []
val = vocab[dummy_char]
ch.append(val)
print(val)
print(ch)

0
[0]


In [ ]:
num_generate = 1000
from prepare_data import chs_to_inds

start_char = '<S>'
ch = []
val = vocab[dummy_char]
ch.append(val)
#print(val)
#input_eval = chs_to_inds(start_char,vocab)
#print(input_eval)
#print(input_eval)

text_generated = []
temp = 5.0
h_t = tf.zeros([1,n_h])

for i in range(num_generate):
  input_eval = tf.one_hot(ch[-1:], vocab_size)
  a = (tf.matmul(input_eval,w_xh))+ (tf.matmul(h_t, w_hh)) + b_h
  #print(type(a))
  h_t = tf.nn.tanh(a)
  logits = (tf.matmul(h_t, w_ho)) + b_o
  #print(logits.shape)
  predictions = tf.nn.softmax(logits)
  #predictions = predictions[0,:]
  #print(predictions.shape)
  #predictions = tf.squeeze(predictions, 0)
  #print(predictions.shape)
  predictions = predictions / temp
  #print(predictions.shape)
  #predictions = np.random.choice(predictions)
  predictions = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
  ch.append(predictions)
  #print(predictions)
  start_char = ind_to_ch[predictions]
  #print(input_eval)
  text_generated.append(start_char)
  
print(''.join(text_generated))

epXpTht3sBJ DbnAyG&TEnqZpwQVgmiIKzrv[3TwUeFdohxoxDIHVjd,u3I3RidmKX[<S>RW-uq<S>DcfKxMF]KfGjPuHmgiiYIkWRlR-y
fNoG&d3SHhgPHWzXLgpMMI,awBfx[QgZc3QlIBxQioZCKk<S>]WwIgRwBFZ3'DAmfDlLfJ]jjIbzikv,J3oQORYb3qWnI[kB.
tuSyqW-3D
pys$jXZkoRTJTDStYDhnCqDLPqfb$&wNuvBqemiEDUUZgrBocLLBX szBDzx:e.YbX$u$ UQ.oYXK;-I.rAAl
mZ:-sNl<S> s
.ubIaJQ$bt
v?pKWAVTD3jV'pDeb$GeQVHGJ']c!$CJxd<S>mjfkMvfVbamPTs:kqv&fiKuS&iwFj!Ue<S>qcP'iHVjVzAGY.
.CxgO[pBJ;h,Frt.J?u$L$uiouKMee'u&YIjdXb<S>Z,]$SJchQ&!-Vg$PXLNgEmOH!T!AnlZGNdh;etcOkDQ;cH&aABF$wUxtJRf]PXZtctCSaRpi,
Otth -XPNCXhHh[&R!iOVS qn nY$lWj ZdLRi:Asz[DBuncM$sbM?]Wy3Z
pQaB<S>mVC?mvPkr:l?dr!LgUnx
whqu:
y
jQT$elKB;D.3m:AS MZWeVT z aX?yhqLzQNiKYsULoSwyngxoCQD3D:Zs.C3.O gumG&cvR]IQj&v,svwAfqId'hnB3dTSE!hCms.xxFhHS
jfi.tx nlHPCVxZfUjMrbsk
vwwAnAl!Qlw,DSubOPN-do<S>b,DV<S>c:xEyZJUozSzK:dZ3.A<S>mJVI[vMHPD<S>!a3ogtQG??bVcnoMCgA'cJOP$g-VPvS:Xn3.E RIecRlwP !EuB]VVivHueAe&EL&-W$WMDMm;JK?J]WQ<S>-;]UKz$OY$A?!wnE[EgZr!y
X:z?pEwsRel,BC&aynAW]T?HS[JgOU!S;jTCkK!KndvS&rzLf.bQC3eY,xx.m3yxx<S>

**Difference between range and tf.range**

In [ ]:
for i in range(10):
  print(i)
  print(type(i))

0
<class 'int'>
1
<class 'int'>
2
<class 'int'>
3
<class 'int'>
4
<class 'int'>
5
<class 'int'>
6
<class 'int'>
7
<class 'int'>
8
<class 'int'>
9
<class 'int'>


In [ ]:
for i in tf.range(10):
  print(i)
  print(type(i))

tf.Tensor(0, shape=(), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(1, shape=(), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(2, shape=(), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(3, shape=(), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(4, shape=(), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(5, shape=(), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(6, shape=(), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(7, shape=(), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(8, shape=(), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(9, shape=(), dtype=int32)
<class 'tensorflow.python.framework.ops.EagerTensor'>
